<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inputs" data-toc-modified-id="Inputs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inputs</a></span></li><li><span><a href="#Script" data-toc-modified-id="Script-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script</a></span></li><li><span><a href="#Execute" data-toc-modified-id="Execute-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Execute</a></span></li></ul></div>

# Batch process

This notebook bundles all of the methodology covered in [notebook 1-fix-link](#) to process one year of albedos per cycle. Import requirements and apply some settings:

## Inputs

In [1]:
import glob

glob.glob("data/*")

['data/MCD43A1-Florida-2016-request.json',
 'data/MCD43A1.2017.nc',
 'data/MCD43A1-Florida-2015-request.json',
 'data/skyl_lut.dat',
 'data/MCD43A1-Florida-2018-request.json',
 'data/MCD43A1-Florida-2017-request.json',
 'data/MCD43A1.2015.nc',
 'data/MCD43A1.2018.nc',
 'data/MCD43A1.2016.nc']

In [2]:
ncin = 'data/MCD43A1.2016.nc'
ncout = "Albedos2016.nc"

bands = [
    "BRDF_Albedo_Parameters_Band1",
    "BRDF_Albedo_Parameters_Band2",
    "BRDF_Albedo_Parameters_Band3",
    "BRDF_Albedo_Parameters_Band4",
    "BRDF_Albedo_Parameters_Band5",
    "BRDF_Albedo_Parameters_Band6",
    "BRDF_Albedo_Parameters_Band7",
    "BRDF_Albedo_Parameters_nir",
    "BRDF_Albedo_Parameters_shortwave",
    "BRDF_Albedo_Parameters_vis",
]

## Script

In [3]:
## processing -->>
from pyproj import Proj, transform
from math import radians, cos
from io import StringIO
import xarray as xr   
import pandas as pd
import numpy as np
import datetime
import sys
import os

import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

##############################################################################
# COORDINATE + SZA FUNCTIONS
##############################################################################


def get_proj(crs):
    """Get srs parameters as proj4 string."""
    
    getpar = lambda a: str(crs.attrs[a])
    return(Proj(" ".join([
        "+proj=sinu",
        "+lon_0="+getpar("longitude_of_central_meridian"),
        "+x_0="+getpar("false_easting"),
        "+y_0="+getpar("false_northing"),
        "+a="+getpar("semi_major_axis"),
        "+b="+getpar("semi_minor_axis"),
        "+units="+"meter +no_defs"])))


# get lat and lon 2d arrays
def get_latlon(ds, inproj, outproj):
    """ """
    
    xx, yy = np.meshgrid(ds.x.data, ds.y.data)
    lon1d, lat1d = transform(
        inproj, 
        outproj, 
        xx.flatten(), 
        yy.flatten())
    lon2d, lat2d = lon1d.reshape(xx.shape), lat1d.reshape(yy.shape)
    
    return(xx, yy, lon1d, lat1d, lon2d, lat2d)


def get_solar_zenith(doy, latitude, ndoy=365):
    """ """
    declination = cos(radians((doy+10)*(360/ndoy)))*-23.45
    altitude = 90 - latitude + declination
    zenith = 90 - altitude
    return(zenith)


def sza_eval(doy, lat):
    """Convert CF to Python datetime."""
    func = lambda l: get_solar_zenith(doy, l)
    return(xr.apply_ufunc(func, lat))


# ----------------------------------------------------------------------------
# 2d lat and lon arrays
# ----------------------------------------------------------------------------


def get_coordinates(ds):
    """ """

    inproj = get_proj(ds.crs)
    outproj = Proj(init="epsg:4326")

    xx, yy, lon1d, lat1d, lon2d, lat2d = get_latlon(ds, inproj, outproj)

    latatts = dict(
        standard_name="latitude",
        long_name="latitude coordinate",
        units="degrees_north")

    ds.coords["lat"] = xr.DataArray(
        data=lat2d, 
        coords=[ds.y, ds.x], 
        dims=["y", "x"], 
        attrs=latatts)

    lonatts = dict(
        standard_name="longitude",
        long_name="longitude coordinate",
        units="degrees_east")

    ds.coords["lon"] = xr.DataArray(
        data=lon2d, 
        coords=[ds.y, ds.x], 
        dims=["y", "x"], 
        attrs=lonatts)
    
    return(ds)


# ----------------------------------------------------------------------------
# Solar zenith angles
# ----------------------------------------------------------------------------


def run_sza(ds):
    """ """

    sza = xr.DataArray(
        data=np.dstack([sza_eval(t.dt.dayofyear, ds.lat) for t in ds.time]), 
        coords=[ds.y, ds.x, ds.time],       # note that we reorder coords in
        dims=["y", "x", "time"],            # dims argument to match others
        attrs=dict(
            units="degree",
            standard_name="solar zenith angle",
            long_name="solar zenith angle"))
    sza.name = "solar_zenith_angle"
    sza = sza.transpose("time", "y", "x")
    
    return(sza)


##############################################################################
# ALBEDO FUNCTIONS
##############################################################################

albedo_attributes = dict(
    _FillValue=32767,
    grid_mapping="crs",
    valid_min=0,
    valid_max=32766,
    units="reflectance, no units",
    scale_factor_err=0.0,
    add_offset_err=0.0,
    calibrated_nt=5,
    scale_factor=0.001,
    add_offset=0.0)

# ----------------------------------------------------------------------------
# Black sky albedo
# ----------------------------------------------------------------------------


def fBSA(param1, param2, param3, sza):
    """ """
    #s = Degrees2Radians*sza              # convert to radians
    s = np.radians(sza)
    func = lambda p1, p2, p3: (
        p1*( 1.0      +  0.0     *(s**2) + 0.0     *(s**3)) +  # Isotropic
        p2*(-0.007574 + -0.070987*(s**2) + 0.307588*(s**3)) +  # RossThick
        p3*(-1.284909 + -0.166314*(s**2) + 0.041840*(s**3)))   # LiSparseR
    
    return(xr.apply_ufunc(func, param1, param2, param3))


# ----------------------------------------------------------------------------
# White sky albedo
# ----------------------------------------------------------------------------


def fWSA(param1, param2, param3):
    """ """
    
    func = lambda p1, p2, p3: (
        p1* 1.0       +           # Isotropic
        p2* 0.189184  +           # RossThick
        p3*-1.377622 )            # LiSparseR  
    
    return(xr.apply_ufunc(func, param1, param2, param3))


# ----------------------------------------------------------------------------
# Blue sky albedo 
# ----------------------------------------------------------------------------


def lookup(sza, luc):
    """ """    
    lfunc = lambda s: luc.iloc[s].values
    return(xr.apply_ufunc(lfunc, abs(sza).round(),))


def fALB(wsa, bsa, lookup):
    """Vectorize albedo polynomials over two 3d arrays."""
    afunc = lambda w,b,l: (w*l)+(b*(1-l))
    return(xr.apply_ufunc(afunc, wsa, bsa, lookup))


# ----------------------------------------------------------------------------
# Add each band's lookup table to the dataset as a variable attribute.
# ----------------------------------------------------------------------------

with open("data/skyl_lut.dat", "r") as f:
    tab = f.read().replace("  ", " ")
con, mar = [t.split("Band") for t in tab.split("Aerosol_type: ")[1:]]
get_lut = lambda s: pd.read_csv(
    StringIO(s),
    index_col="S&O",
    skiprows=1,
    sep=" ")

# ----------------------------------------------------------------------------
# Add each band's lookup table to the dataset as a variable attribute.
# ----------------------------------------------------------------------------

get_band_name = lambda ds:[v for v in ds.variables if all([
    "param" in ds[v].dims, v!="param"])][0]


def run_albedo(ds, sod="0.02"): 
    """ """
    
    band_name = get_band_name(ds)
    band = ds[band_name]
    param1 = band.sel(param=0)
    param2 = band.sel(param=1)
    param3 = band.sel(param=2)
    sza = ds["solar_zenith_angle"]
    
    # [1/3] Calculating black sky albedo
    bsa = fBSA(param1, param2, param3, sza)
    bsa.name = "black_sky_albedo"
    bsa.attrs = albedo_attributes
    bsa.attrs.update(dict(long_name="black_sky_albedo"))
        
    # [2/3] Calculating white sky albedo
    wsa = fWSA(param1, param2, param3)
    wsa.name = "white_sky_albedo"
    wsa.attrs = albedo_attributes
    wsa.attrs.update(dict(long_name="white_sky_albedo"))

    luv = lookup(
        sza.data.flatten(), 
        band.attrs["lut"][sod]).reshape(sza.shape)
    
    lu = xr.DataArray(
        data=luv, 
        coords=[sza.time, sza.y, sza.x],
        dims=["time", "y", "x"],
        attrs=dict(
            units="unitless",
            long_name="lookup value"))

    # [3/3] Calculating blue sky albedo
    alb = fALB(wsa, bsa, lu)
    alb.name = "blue_sky_albedo"
    alb.attrs = albedo_attributes
    alb.attrs.update(dict(long_name="blue_sky_albedo"))
    
    return(xr.Dataset({
        "blue_sky_albedo": alb, 
        "black_sky_albedo": bsa, 
        "white_sky_albedo": wsa}))

## Execute

This follows the `pandas` and `xarray` *groupby: Split-Apply-Combine*:
http://xarray.pydata.org/en/stable/groupby.html

In [4]:
# open netcdf
ds = xr.open_dataset(ncin)
ds = ds.rename({
    "Num_Parameters": "param", 
    "xdim": "x", 
    "ydim": "y"})

# add lat,lon coordinate arrays
ds = get_coordinates(ds)

# get sza arrays
ds["solar_zenith_angle"] = ds.groupby('time.month').apply(run_sza)
ds[["lat", "lon", "solar_zenith_angle"]].to_netcdf(
    ncout, 
    mode="w", 
    unlimited_dims=["time"],
    encoding={
        "lat": dict(zlib=True, complevel=5),
        "lon": dict(zlib=True, complevel=5),
        "solar_zenith_angle": dict(zlib=True, complevel=5)})

# add lookup table (pandas.DataFrame) as attribute
for i, band in enumerate(bands):
    ds[band].attrs.update({"lut": get_lut(con[i+1])})
    
# loop over bands
for i, band in enumerate(bands):
    print("["+str(i+1)+"/"+str(len(bands))+"]\t--- Processing: "+band) 
    band_parameters = ds[[band, "solar_zenith_angle"]]
    band_result = band_parameters.groupby("time.month").apply(run_albedo)
    band_result.to_netcdf(
        ncout, 
        mode="a", 
        group=band,
        unlimited_dims=["time"],
        encoding={
            "black_sky_albedo": dict(zlib=True, complevel=5),
            "white_sky_albedo": dict(zlib=True, complevel=5),
            "blue_sky_albedo": dict(zlib=True, complevel=5)})

OSError: [Errno -101] NetCDF: HDF error: b'/home/jack/git/florida_usgs_albedo_evapotranspiration/data/MCD43A1.2016.nc'